In [1]:
import ciropt as co
import cvxpy as cp

In [2]:
import numpy as np

In [3]:
L_smooth = 1 #np.inf
mu = 0.0001

Inductance = 1
R = 1

n_func = 3
# solver = "ipopt"
solver = "ipopt_canonical"
# solver = "ipopt_canonical_X"
# solver = "cvx_dccp"
# solver = "qcqp_sni"
# solver = "bisection_b"

# Ciropt problem

In [4]:
problem = co.admm_consensus(n_func, mu, L_smooth, R, Inductance)
problem.obj = problem.b + problem.d * 0.5

# bounds = {  'alpha': {"ub": 0, "lb": 0},
#             'beta': {"ub": 1, "lb": 1},}

# res, sol = problem.solve(solver=solver, bounds=bounds, verbose=False)[:2]
res, model = problem.solve(solver=solver, verbose=False)[:2]
ca_vars = problem.vars

dim_G=12, dim_F=9


In [5]:
res

{'b': 2.6469583917990156,
 'h': 1.0628168789198864,
 'd': 3.969310672332824,
 'alpha': -4.7100215574094157e-14,
 'beta': -0.06529761357478259,
 'gamma': 1.9847867916634503}

# PEP verification

In [8]:
res['b'] -= 0.1
print(res["b"])

2.5469583917990155


In [9]:
problem = co.admm_consensus(n_func, mu, L_smooth, R, Inductance, params=res)
diff = problem.solve(verbose=0, solver=cp.MOSEK)
print(f"{diff=}")

diff=-1.0851254191268474e-09
